In [ ]:
!pip install pysad
!pip install combo
from pysad.evaluation.metrics import AUROCMetric
from pysad.utils.data import Data
from pysad.models.integrations import ReferenceWindowModel

from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.transform.probability_calibration import ConformalProbabilityCalibrator
from pysad.transform.postprocessing import AveragePostprocessor
from pyod.models.loci import LOCI

In [ ]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import col
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'])

spark = SparkSession.builder.appName("Test").getOrCreate()
sdf = spark.createDataFrame(df) 
sdf.show(5)

Local Correlation Integral.

LOCI is highly effective for detecting outliers and groups of outliers ( a.k.a.micro-clusters), which offers the following advantages and novelties: (a) It provides an automatic, data-dictated cut-off to determine whether a point is an outlier—in contrast, previous methods force users to pick cut-offs, without any hints as to what cut-off value is best for a given dataset. (b) It can provide a LOCI plot for each point; this plot summarizes a wealth of information about the data in the vicinity of the point, determining clusters, micro-clusters, their diameters and their inter-cluster distances. 

In [87]:
from influxdb_client.client.write_api import SYNCHRONOUS
import influxdb_client
from pysad.transform.probability_calibration import GaussianTailProbabilityCalibrator
token = 'FFcw-DFs__xLrJSeSoGmir_boRtccVNboSiLo6PZvFQ5IpS_gQwx_YLQffZSK0DkJDIcjpiMdXXUGwn1K5eZDw=='
org = 'changeme'
bucket = 'loci'

# Define the model
model = LOCI()

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    stream = np.array(sdf.select("LIT 301", "FIT 401").collect())
    model = ReferenceWindowModel(model_cls=LOCI,  window_size=500, sliding_size=200)
    preprocessor = InstanceUnitNormScaler()
    
    # Normalizer
    postprocessor = AveragePostprocessor()  #  average postprocessor
   # calibrator = ConformalProbabilityCalibrator(window_size=6400)
    calibrator = GaussianTailProbabilityCalibrator(window_size=6400)
    y_pred = []
    for x in stream:
        X = preprocessor.fit(x)
        anomaly_score = model.fit_score_partial(x)
        anomaly_score = postprocessor.fit_transform_partial(anomaly_score)
        calibrated_score = calibrator.fit_transform_partial(anomaly_score)
        p = influxdb_client.Point("loci").field("Anomaly Score", calibrated_score)
        write_api.write(bucket, org, p) 

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 